<a href="https://colab.research.google.com/github/YanumulaRohith/Ai-ML_Externship/blob/main/WeedDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from PIL import UnidentifiedImageError

# Replace these paths with the paths to your full dataset
full_data_dir = r'/content/drive/MyDrive/Student Projects_503629076'
train_data_dir = r'/content/sample_data/train'
val_data_dir = r'/content/sample_data/val'
test_data_dir = r'/content/sample_data/test'

input_shape = (224, 224)  # Input image dimensions for EfficientNetB0

# Create directories for train, validation, and test data
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(val_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)

# Move a percentage of the full dataset to the train_data_dir
train_split_percentage = 0.7
val_split_percentage = 0.15
test_split_percentage = 0.15

def handle_image_error(image, src_path):
    try:
        return image
    except UnidentifiedImageError:
        print(f"Removing problematic image: {src_path}")
        os.remove(src_path)
        return None
    except Exception as e:
        print(f"Error handling image {src_path}: {e}")
        return None


for class_name in os.listdir(full_data_dir):
    class_dir = os.path.join(full_data_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    images = os.listdir(class_dir)
    random.shuffle(images)
    total_images = len(images)
    train_split_idx = int(train_split_percentage * total_images)
    val_split_idx = int((train_split_percentage + val_split_percentage) * total_images)

    for i, image in enumerate(images):
        src_path = os.path.join(class_dir, image)
        filename = f"{i}_{image}"
        if i < train_split_idx:
            dst_dir = os.path.join(train_data_dir, class_name)
        elif i < val_split_idx:
            dst_dir = os.path.join(val_data_dir, class_name)
        else:
            dst_dir = os.path.join(test_data_dir, class_name)

        dst_path = os.path.join(dst_dir, filename)
        try:
            shutil.copy(src_path, dst_path)
        except Exception as e:
            print(f"Error copying image {filename}: {e}")
            pass  # Skip problematic images silently

# Define data generators for training, validation, and testing
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load EfficientNetB0 with pre-trained ImageNet weights (exclude top layer)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(input_shape[0], input_shape[1], 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for rice plant disease classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model with early stopping and dropout
epochs = 20

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy: {test_accuracy}")


Found 1750 images belonging to 3 classes.
Found 371 images belonging to 3 classes.
Found 378 images belonging to 3 classes.
Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_6 (Rescaling)     (None, 224, 224, 3)          0         ['input_18[0][0]']            
                                                                                                  
 normalization_3 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_6[0][0]']         
 tion)                                                                                            
                                                                  

In [31]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from PIL import UnidentifiedImageError

# Replace these paths with the paths to your full dataset
full_data_dir = r'/content/drive/MyDrive/Student Projects_503629076'
train_data_dir = r'/content/sample_data/train'
val_data_dir = r'/content/sample_data/val'
test_data_dir = r'/content/sample_data/test'

input_shape = (224, 224)  # Input image dimensions for InceptionV3

# Create directories for train, validation, and test data
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(val_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)

# Move a percentage of the full dataset to the train_data_dir
train_split_percentage = 0.7
val_split_percentage = 0.15
test_split_percentage = 0.15

def handle_image_error(image, src_path):
    try:
        return image
    except UnidentifiedImageError:
        print(f"Removing problematic image: {src_path}")
        os.remove(src_path)
        return None
    except Exception as e:
        print(f"Error handling image {src_path}: {e}")
        return None


for class_name in os.listdir(full_data_dir):
    class_dir = os.path.join(full_data_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    images = os.listdir(class_dir)
    random.shuffle(images)
    total_images = len(images)
    train_split_idx = int(train_split_percentage * total_images)
    val_split_idx = int((train_split_percentage + val_split_percentage) * total_images)

    for i, image in enumerate(images):
        src_path = os.path.join(class_dir, image)
        filename = f"{i}_{image}"
        if i < train_split_idx:
            dst_dir = os.path.join(train_data_dir, class_name)
        elif i < val_split_idx:
            dst_dir = os.path.join(val_data_dir, class_name)
        else:
            dst_dir = os.path.join(test_data_dir, class_name)

        dst_path = os.path.join(dst_dir, filename)
        try:
            shutil.copy(src_path, dst_path)
        except Exception as e:
            print(f"Error copying image {filename}: {e}")
            pass  # Skip problematic images silently

# Define data generators for training, validation, and testing
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda image: handle_image_error(image, src_path)
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load InceptionV3 with pre-trained ImageNet weights (exclude top layer)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(input_shape[0], input_shape[1], 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for rice plant disease classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model with early stopping and dropout
epochs = 1
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test accuracy: {test_accuracy}")


Found 1947 images belonging to 3 classes.
Found 413 images belonging to 3 classes.
Found 431 images belonging to 3 classes.
Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_1410 (Conv2D)        (None, 111, 111, 32)         864       ['input_20[0][0]']            
                                                                                                  
 batch_normalization_1410 (  (None, 111, 111, 32)         96        ['conv2d_1410[0][0]']         
 BatchNormalization)                                                                              
                                                                  

In [33]:
import pickle
pickle.dump(model,open('WeedDetection.pkl','wb'))